In [1]:
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import peakutils
import json

In [2]:
hk_directory = "/home/ubuntu/UPS2_allion/hk"
feature_id = 19869
minimum_peak_correlation = 0.6
hk_filename = "{}/feature-{}-correlation-{}.hk".format(hk_directory, feature_id, minimum_peak_correlation)
hk_results_df = pd.read_table(hk_filename, skiprows=1, header=None, names=['monoisotopic_mass','charge','intensity','base_isotope_peak','analysis_window','deprecated','modifications','correlation'])

In [3]:
len(hk_results_df)

32

In [4]:
hk_results_df.head()

,monoisotopic_mass,charge,intensity,base_isotope_peak,analysis_window,deprecated,modifications,correlation
P,989.3841,3,8266,330.802,328.9110-331.9370,0.0,_,0.7213
P,355.2217,1,29952,356.229,356.1090-359.2230,0.0,_,0.9651
P,716.4314,2,36674,359.223,358.1100-360.9670,0.0,_,0.9926
P,1163.3181,3,16391,388.780,387.0930-391.0220,0.0,_,0.9602
P,1236.0891,3,10393,413.037,411.8920-417.0080,0.0,_,0.9190


In [5]:
db_name = '/home/ubuntu/UPS2_allion/UPS2_allion-features-19738-20833.sqlite'
db_conn = sqlite3.connect(db_name)
ms2_peaks_df = pd.read_sql_query("select feature_id,peak_id,centroid_mz from ms2_peaks where feature_id={}".format(feature_id), db_conn)
db_conn.close()

In [6]:
len(ms2_peaks_df)

27999

In [7]:
ms2_peaks_df.head()

,feature_id,peak_id,centroid_mz
0,19869,1,600.334
1,19869,2,599.999
2,19869,3,809.449
3,19869,4,600.668
4,19869,5,595.353


For each fragment in the HK output, find the ms2 peak it originated from. The HK 'base_isotope_peak' should match the ms2 peak's 'centroid_mz'.

In [8]:
hk_results_df = pd.merge(hk_results_df, ms2_peaks_df, left_on=['base_isotope_peak'], right_on=['centroid_mz'])

In [9]:
hk_results_df['intensity_peak_id'] = hk_results_df['intensity'].astype(str) + "." + hk_results_df['peak_id'].map('{0:05d}'.format)

In [10]:
hk_results_df.head()

,monoisotopic_mass,charge,intensity,base_isotope_peak,analysis_window,deprecated,modifications,correlation,feature_id,peak_id,centroid_mz,intensity_peak_id
0,989.3841,3,8266,330.802,328.9110-331.9370,0.0,_,0.7213,19869,22977,330.802,8266.22977
1,355.2217,1,29952,356.229,356.1090-359.2230,0.0,_,0.9651,19869,6283,356.229,29952.06283
2,716.4314,2,36674,359.223,358.1100-360.9670,0.0,_,0.9926,19869,3966,359.223,36674.03966
3,1163.3181,3,16391,388.780,387.0930-391.0220,0.0,_,0.9602,19869,14043,388.780,16391.14043
4,1236.0891,3,10393,413.037,411.8920-417.0080,0.0,_,0.9190,19869,13939,413.037,10393.13939


In [12]:
len(hk_results_df)

32

In [14]:
hk_results_df[hk_results_df.monoisotopic_mass == 989.3841]

,monoisotopic_mass,charge,intensity,base_isotope_peak,analysis_window,deprecated,modifications,correlation,feature_id,peak_id,centroid_mz,intensity_peak_id
0,989.3841,3,8266,330.802,328.9110-331.9370,0.0,_,0.7213,19869,22977,330.802,8266.22977
